In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.abspath('..'),
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)
print(sys.path)

# Imports
import numpy as np
import matplotlib.pyplot as plt
from electronTransportCode.SimOptions import WaterPhantomSimulation
from electronTransportCode.SimulationDomain import SimulationDomain
from electronTransportCode.MCParticleTracer import AnalogParticleTracer, MCParticleTracer
from electronTransportCode.MCEstimator import DoseEstimator
from electronTransportCode.ProjectUtils import ERE
from electronTransportCode.ParticleModel import SimplifiedEGSnrcElectron
from electronTransportCode.ProjectUtils import E_THRESHOLD, Z_WATER_EFF
from electronTransportCode.Material import WaterMaterial, Material

['/home/klaas/Documents/KUL/thesis/electronTransportCode/numericalExperiments/waterPhantom', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/klaas/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/home/klaas/Documents/codes/yt', '/usr/lib/python3/dist-packages', '/home/klaas/Documents/KUL/thesis/electronTransportCode']


In [2]:
import pickle

with open('data/particleTracer.pkl', 'rb') as file:
    particleTracer = pickle.load(file)
    
with open('data/doseEstimator.pkl', 'rb') as file:
    doseEstimator = pickle.load(file)
    
simDomain = particleTracer.simDomain
particle = particleTracer.particle

In [ ]:
# Plot energy deposition
energyDeposition_ERE = np.reshape(doseEstimator.scoreMatrix, (simDomain.ybins, simDomain.xbins) )

fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.matshow(energyDeposition_ERE, extent=[-2.5, 7.5, -2.5, 7.5], origin='lower')
ax.set_xticks(np.linspace(-2.5, 7.5, 5))
ax.set_yticks(np.linspace(-2.5, 7.5, 5))
ax.xaxis.tick_bottom()
ax.set_xlabel('x [cm]')
ax.set_ylabel('y [cm]')
plt.show()